In [8]:
import sys
sys.path.append("../src")

import plotly.graph_objs as go
import plotly.offline as py

import utilities as u
import constants as c

In [10]:
import dash_app

In [11]:
aux = dash_app.create_dash_app()

In [15]:
aux.

[]

In [16]:
aux.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


In [22]:
aux.config

{'requests_pathname_prefix': '/',
 'routes_pathname_prefix': '/',
 'suppress_callback_exceptions': False,
 'supress_callback_exceptions': True}